In [1]:
import os
os.chdir("../..")
import datetime
import math
import pandas as pd
import numpy as np
from scripts.python.PdfParse import *
from scripts.python.ts_utils import *
from scripts.python.utils import *

## Data Wrangling
### VU's aviation data

In [2]:
aviation_path = os.getcwd() + "/data/tourism/aviation_seats_flights_pic.xlsx"
aviation = pd.read_excel(aviation_path)
aviation.head(5)

,Country,ISO,Region,Date,Aircraft_type,Seats_arrivals_domestic,Seats_arrivals_interregional,Seats_arrivals_intraregional,Seats_arrivals_intl,Seats_arrivals_total,Available_seat_kilometers,Number_of_flights_domestic,Number_of_flights_interregional,Number_of_flights_intraregional,Number_of_flights_intl,Number_of_flights_total
0,Fiji,FJ,East Asia & Pacific,2019-01-01,passenger,839,273,3480,3753,4592,1.430416e+07,8,1,10,11,19
1,Fiji,FJ,East Asia & Pacific,2019-01-02,passenger,974,313,3471,3784,4758,1.495610e+07,8,1,10,11,19
2,Fiji,FJ,East Asia & Pacific,2019-01-03,passenger,1190,443,3675,4118,5308,1.592143e+07,10,2,12,14,24
3,Fiji,FJ,East Asia & Pacific,2019-01-04,passenger,831,586,3159,3745,4576,1.457334e+07,7,2,12,14,21
4,Fiji,FJ,East Asia & Pacific,2019-01-05,passenger,744,273,4752,5025,5769,1.773449e+07,7,1,12,13,20


In [3]:
vu_avi = (aviation[(aviation.ISO == "VU") & (aviation.Aircraft_type == "passenger")]
          .reset_index()
          .drop("index", axis=1)
          [["Date", "Number_of_flights_intl", "Number_of_flights_total",
            "Seats_arrivals_intl", "Seats_arrivals_total"]])

dates = pd.DataFrame(pd.date_range(start="2019-01-01",
                                   end="2022-10-16"), columns=["Date"])

vu_avi = dates.merge(vu_avi, how="left", on="Date")
vu_avi["Date"] = pd.to_datetime(vu_avi["Date"])
vu_avi["Month"], vu_avi["Year"] = vu_avi["Date"].dt.month, vu_avi["Date"].dt.year
vu_avi.head(5)

,Date,Number_of_flights_intl,Number_of_flights_total,Seats_arrivals_intl,Seats_arrivals_total,Month,Year
0,2019-01-01,4.0,4.0,552.0,552.0,1,2019
1,2019-01-02,4.0,4.0,584.0,584.0,1,2019
2,2019-01-03,3.0,3.0,408.0,408.0,1,2019
3,2019-01-04,4.0,4.0,674.0,674.0,1,2019
4,2019-01-05,4.0,4.0,652.0,652.0,1,2019


In [4]:
vu_avi_19_22 = vu_avi.groupby(by=["Year", "Month"]).sum().reset_index()
vu_avi_19_22.head(5)

,Year,Month,Number_of_flights_intl,Number_of_flights_total,Seats_arrivals_intl,Seats_arrivals_total
0,2019,1,123.0,123.0,18568.0,18568.0
1,2019,2,94.0,97.0,13680.0,14190.0
2,2019,3,101.0,101.0,14774.0,14774.0
3,2019,4,112.0,112.0,17071.0,17071.0
4,2019,5,111.0,111.0,16764.0,16764.0


### VU's Parsed Files

In [5]:
# def generate_time(df: pd.DataFrame,
#                   month_var: str,
#                   year_var: str):

#     import datetime

#     for idx in df.index:
#         time = df[month_var][idx]

#         if type(time) == str and len(time) != 0:
#             try:
#                 month_num = datetime.datetime.strptime(time, "%b").month
#                 if month_num == 1:
#                     year = df[year_var][idx]
#                     break
#             except:
#                 pass

#     year_lst, length = df[year_var].to_list(), len(df[year_var])
#     for i in range(idx, length):
#         year_lst[i] = (i-idx) // 12 + year

#     return year_lst, idx

In [6]:
parsed_folder = os.getcwd() + "/data/tourism/vanuatu/byorigin/"
filepaths = os.listdir(parsed_folder)
filepaths = [parsed_folder + file for file in filepaths]

In [194]:
vu_parsed = pd.DataFrame()

for file in filepaths:
    filename = file.split("/")[-1]
    match = re.search(r'\d{4}', filename)
    print(f"{filename} has started.")
    df = pd.read_csv(file)
    df = (df.replace(r"\[p\]", "", regex=True)
            .drop("Unnamed: 0", axis=1)
            .fillna(method="ffill"))
    df = df.rename({"Quarter": "Month",
                    "Chine": "China",
                    "Japon": "Japan",
                    "Australie": "Australia",
                    "Caledonia": "New Caledonia",
                    "Zealand": "New Zealand",
                    "America": "North America",
                    "Stated": "Not Stated",
                    "Countries": "Other Countries"}, axis=1)
    if match:
        df["fileyear"] = match.group()
    if "Month" and "Year" in df.columns:
        vu_parsed = (pd.concat([vu_parsed, df], axis=0)
                       .reset_index()
                       .sort_values(by="Year", ascending=True)
                       .drop("index", axis=1))
    else:
        pass
        print(f"{filename} has no Year or Month Column.")
    print(f"{filename} has ended.")

IVA_3_March_2022.csv has started.
IVA_3_March_2022.csv has ended.
IVA_7_July_2022.csv has started.
IVA_7_July_2022.csv has ended.
IVA_4_April_2022.csv has started.
IVA_4_April_2022.csv has ended.
IVA_1_January_2022.csv has started.
IVA_1_January_2022.csv has ended.
IVA_12_December-English_2019.csv has started.
IVA_12_December-English_2019.csv has ended.
IVA_5_May_2022.csv has started.
IVA_5_May_2022.csv has ended.
IAS_12_December_2018.csv has started.
IAS_12_December_2018.csv has ended.
IAS_12_Dececember_2017.csv has started.
IAS_12_Dececember_2017.csv has ended.
IVA_12_Dec_2020.csv has started.
IVA_12_Dec_2020.csv has ended.
2013-TM-12-December_News.csv has started.
2013-TM-12-December_News.csv has ended.
Tou12_December_2006.csv has started.
Tou12_December_2006.csv has ended.
TM12_December_2008_News.csv has started.
TM12_December_2008_News.csv has ended.
TM12_December_2009_News.csv has started.
TM12_December_2009_News.csv has ended.
IVA_12_December_2021.csv has started.
IVA_12_Decembe

In [191]:
month_equv = dict()

for i in range(1, 13):
    month_abbre = datetime.date(1900, i, 1).strftime('%b')
    month_full = datetime.date(1900, i, 1).strftime('%B')
    month_equv.update({month_full: i, month_abbre: i})

print(month_equv)

{'January': 1, 'Jan': 1, 'February': 2, 'Feb': 2, 'March': 3, 'Mar': 3, 'April': 4, 'Apr': 4, 'May': 5, 'June': 6, 'Jun': 6, 'July': 7, 'Jul': 7, 'August': 8, 'Aug': 8, 'September': 9, 'Sep': 9, 'October': 10, 'Oct': 10, 'November': 11, 'Nov': 11, 'December': 12, 'Dec': 12}


In [195]:
vu_parsed = (vu_parsed
             .replace(r"\*", "", regex=True)
             .drop_duplicates()
             .reset_index()
             .drop("index", axis=1)
             .fillna(0))

vu_parsed["month"] = vu_parsed["Month"].map(month_equv)


# Check the failed mappings
for idx in vu_parsed.index:
    month = vu_parsed["month"][idx]
    if math.isnan(month) == False:
        pass
    else:
        print(idx, vu_parsed["Year"][idx], vu_parsed["Month"][idx])


# vu_parsed = vu_parsed.iloc[:-1].drop("Month", axis=1)
# vu_parsed.to_csv(
#     os.getcwd() + "/data/tourism/vu_monthly_visitor.csv", encoding="utf-8")

1 2002.0 0
2 2003.0 4th
3 2003.0 3rd
5 2003.0 1st
6 2003.0 0
8 2003.0 2nd
10 2004.0 3rd
11 2004.0 1st
13 2004.0 0
18 2004.0 2nd
20 2004.0 4th
30 2005.0 2nd
31 2005.0 0
35 2005.0 3rd
44 2005.0 4th
46 2005.0 1st
131 2009.0 0
152 2010.0 0
154 2010.0 0
164 2011.0 0
165 2011.0 0
176 2011.0 0
192 2012.0 0
195 2012.0 0
199 2012.0 0
201 2012.0 0
227 2013.0 0
232 2013.0 0
233 2013.0 0
236 2013.0 0
246 2013.0 0
250 2013.0 0
255 2014.0 0
258 2014.0 0
259 2014.0 0
262 2014.0 0
267 2014.0 0
285 2014.0 0
290 2015.0 0
308 2015.0 0
312 2015.0 0
314 2015.0 0
319 2015.0 0
322 2015.0 0
326 2016.0 0
335 2016.0 0
343 2016.0 0
344 2016.0 0
345 2016.0 0
346 2016.0 0
348 2016.0 0
352 2017.0 0
353 2017.0 0
356 2017.0 0
359 2017.0 0
373 2017.0 0
375 2017.0 0
378 2018.0 0
386 2018.0 0
399 2018.0 0
401 2018.0 0
402 2018.0 0
404 2019.0 0
410 2019.0 
413 2019.0 0
432 2020.0 0
448 2020.0 0
469 2021.0 0
479 2022.0 Ap


In [211]:
stored_idx = (vu_parsed[vu_parsed.month.isna() == False]
              [vu_parsed.columns[~vu_parsed.columns.isin(["fileyear"])]]
              .drop_duplicates().index)

vu_parsed.iloc[stored_idx].groupby(by="Year").count()

,Month,Australia,New Zealand,New Caledonia,Other PIC,Europe,North America,Japan,China,Other Countries,Total,fileyear,Not Stated,month
Year,,,,,,,,,,,,,,
2002.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2003.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2004.0,12,12,12,12,12,12,12,12,12,12,12,12,12,12
2005.0,13,13,13,13,13,13,13,13,13,13,13,13,13,13
2006.0,13,13,13,13,13,13,13,13,13,13,13,13,13,13
2007.0,13,13,13,13,13,13,13,13,13,13,13,13,13,13
2008.0,13,13,13,13,13,13,13,13,13,13,13,13,13,13
2009.0,12,12,12,12,12,12,12,12,12,12,12,12,12,12
2010.0,12,12,12,12,12,12,12,12,12,12,12,12,12,12


In [213]:
vu_parsed.iloc[stored_idx][vu_parsed.Year==2012].sort_values(by="month")

,Year,Month,Australia,New Zealand,New Caledonia,Other PIC,Europe,North America,Japan,China,Other Countries,Total,fileyear,Not Stated,month
203,2012.0,Jan,5653,757,1458,247,341.0,105,49,0.0,145,8755,2013,0.0,1.0
194,2012.0,Feb,2387,354,814,403,367.0,130,59,0.0,186,4700,2013,0.0,2.0
209,2012.0,Mar,3970,641,707,277,332.0,131,52,0.0,233,6343,2012,0.0,3.0
204,2012.0,Apr,5021,838,991,270,335.0,133,58,0.0,214,7860,2013,0.0,4.0
202,2012.0,May,4091,821,1198,454,410.0,219,59,0.0,222,7474,2013,0.0,5.0
197,2012.0,Jun,6248,1690,726,310,523.0,203,52,0.0,224,9976,2013,0.0,6.0
190,2012.0,Jul,6937,2032,1188,433,633.0,216,95,0.0,236,11770,2013,0.0,7.0
207,2012.0,Aug,6043,2049,928,298,646.0,181,58,0.0,202,10405,2013,0.0,8.0
196,2012.0,Sep,7640,1827,1227,451,462.0,231,59,0.0,285,12182,2013,0.0,9.0
193,2012.0,Oct,5960,1505,1072,334,576.0,216,41,0.0,218,9922,2013,0.0,10.0


In [47]:
vu_bc19 = (vu_parsed[vu_parsed.Year < 2019]
           .drop_duplicates()
           .replace(0, np.nan)
           .dropna(thresh=6))

problem_yr = [2016, 2017]
vu_problem = (vu_bc19[vu_bc19.Year.isin(problem_yr)]
              .sort_values(by=["Year", "month"], ascending=True))
vu_problem

,Year,Australia,New Zealand,New Caledonia,Other PIC,Europe,North America,Japan,China,Other Countries,Total,Not Stated,month
179,2016.0,4495.0,724.0,1060.0,486.0,979.0,276.0,75.0,174.0,210.0,8479.0,NaN,1.0
186,2016.0,5206.0,804.0,2129.0,528.0,570.0,251.0,63.0,317.0,262.0,10130.0,NaN,1.0
188,2016.0,5251.0,767.0,1786.0,481.0,523.0,229.0,61.0,284.0,166.0,9548.0,NaN,1.0
189,2016.0,6425.0,943.0,2087.0,402.0,837.0,335.0,89.0,522.0,293.0,11933.0,NaN,1.0
191,2016.0,4738.0,549.0,1742.0,439.0,602.0,180.0,75.0,339.0,293.0,8957.0,NaN,1.0
171,2016.0,1714.0,310.0,200.0,396.0,1027.0,180.0,67.0,226.0,154.0,4274.0,NaN,2.0
172,2016.0,2060.0,325.0,313.0,267.0,664.0,168.0,44.0,175.0,139.0,4155.0,NaN,3.0
173,2016.0,2749.0,636.0,478.0,344.0,1160.0,178.0,53.0,262.0,223.0,6083.0,NaN,4.0
174,2016.0,2644.0,552.0,632.0,508.0,823.0,190.0,51.0,181.0,161.0,5742.0,NaN,5.0
175,2016.0,5170.0,1197.0,1143.0,402.0,495.0,306.0,70.0,217.0,158.0,9158.0,NaN,6.0


In [24]:
# Data of three years seem to be problematic 2014, 2016, 2017
vu_parsed.drop_duplicates().groupby("Year").count()


vu_problem = (vu_parsed[vu_parsed.Year.isin(problem_yr)]
              .sort_values(by=["Year", "month"], ascending=True))
vu_problem[["Year", "month", "Total"]].drop_duplicates()

,Year,month,Total
136,2014.0,1.0,8447
137,2014.0,2.0,5009
139,2014.0,3.0,6376
152,2014.0,4.0,6804
145,2014.0,5.0,7812
142,2014.0,6.0,10917
141,2014.0,7.0,11446
135,2014.0,8.0,11865
144,2014.0,9.0,11288
138,2014.0,10.0,10533


In [10]:
vu_19_22 = (vu_parsed[vu_parsed.Year >= 2019]
            .dropna(axis=1)
            .drop_duplicates()
            .reset_index()
            .drop("index", axis=1))


months = list()
for month in vu_19_22["Month"]:
    try:
        month = datetime.datetime.strptime(month, "%b").month
        months.append(month)
    except:
        try:
            month = datetime.datetime.strptime(month, "%B").month
            months.append(month)
        except:
            months.append(month)

vu_19_22["Month"] = months

In [11]:
## Manually drop the last row because of the typo
vu_19_22 = vu_19_22[:-1].sort_values(by=["Year", "Month"], ascending=True)
vu_19_22.head(5)

,Year,Month,Australia,New Zealand,New Caledonia,Other PIC,Europe,North America,Japan,China,Other Countries,Total,Not Stated
7,2019.0,1,5251,767,1786,481,523.0,229,61,284.0,166,9548,0.0
5,2019.0,2,2740,503,923,526,456.0,152,57,326.0,152,5835,0.0
9,2019.0,3,3864,611,949,411,413.0,209,71,331.0,167,7026,0.0
4,2019.0,4,5195,1010,1164,507,731.0,180,102,294.0,262,9445,0.0
8,2019.0,5,3919,868,1139,552,806.0,294,60,425.0,296,8359,0.0


In [22]:
# dates = list()
# for year, month in zip(vu_parsed["Year"], vu_parsed["Month"]):
#     date = str(year).replace(".0", "") + "-" + str(month) + "-01"
#     dates.append(date)

# vu_parsed["Date"] = dates
# vu_parsed["Date"] = pd.to_datetime(vu_parsed["Date"])


# from bokeh.palettes import Category20
# from bokeh.plotting import figure, show, output_file
# from bokeh.models import ColumnDataSource, HoverTool, Legend


# output_file('vu_month_by_country.html')
# p = figure(height=600, width=1000,
#            title="Number of Passengers Per Month",
#            x_axis_type="datetime",
#            x_axis_label="Date",
#            y_axis_label="Number of Passengers Per Month",
#            tooltips=[("Date", "$Date"),
#                      ("Country", "$name"),
#                      ("Passenger per month", "@$name")])

# countries = vu_parsed.columns[~vu_parsed.columns.isin(["Year", "Month", "Total", "Date"])].to_list()
# p.add_layout(Legend(), 'right')
# source = ColumnDataSource(vu_parsed)

# for country, color in zip(countries, Category20[12]):
#     # have to use different colnames for y-coords so tooltip can refer to @$name
#     p.line('Date', country, source=source, name=country,
#            legend_label=country, color=color)

# p.legend.label_text_font_size = '9pt'
# p.legend.click_policy = "mute"
# p.legend.location = "top_left"

# show(p)

## EDA
### Correlation and Cross-correlation

In [12]:
vu_merged = vu_avi_19_22.merge(vu_19_22, how="left", on=["Year", "Month"]).dropna()
vu_merged.head(5)

,Year,Month,Number_of_flights_intl,Number_of_flights_total,Seats_arrivals_intl,Seats_arrivals_total,Australia,New Zealand,New Caledonia,Other PIC,Europe,North America,Japan,China,Other Countries,Total,Not Stated
0,2019,1,123.0,123.0,18568.0,18568.0,5251.0,767.0,1786.0,481.0,523.0,229.0,61.0,284.0,166.0,9548.0,0.0
1,2019,2,94.0,97.0,13680.0,14190.0,2740.0,503.0,923.0,526.0,456.0,152.0,57.0,326.0,152.0,5835.0,0.0
2,2019,3,101.0,101.0,14774.0,14774.0,3864.0,611.0,949.0,411.0,413.0,209.0,71.0,331.0,167.0,7026.0,0.0
3,2019,4,112.0,112.0,17071.0,17071.0,5195.0,1010.0,1164.0,507.0,731.0,180.0,102.0,294.0,262.0,9445.0,0.0
4,2019,5,111.0,111.0,16764.0,16764.0,3919.0,868.0,1139.0,552.0,806.0,294.0,60.0,425.0,296.0,8359.0,0.0


In [13]:
from scipy.stats import pearsonr

corr_seats, _ = pearsonr(vu_merged["Seats_arrivals_total"], vu_merged["Total"])
corr_seats_intl, _ = pearsonr(
    vu_merged["Seats_arrivals_intl"], vu_merged["Total"])
print(f"Pearson Correlation between FlightRadar's Seats Arrival and VU's census data is{corr_seats: .4f}.\n",
      f"Pearson Correlation between FlightRadar's # of Seats Arrival (International) and VU's census data is{corr_seats_intl: .4f}.")

Pearson Correlation between FlightRadar's Seats Arrival and VU's census data is 0.7161.
 Pearson Correlation between FlightRadar's # of Flights Arrival and VU's census data is 0.9592.


In [21]:
incl_cols = ["Seats_arrivals_intl", "Seats_arrivals_total", "Total"]
vu_merged[incl_cols].corr()

,Seats_arrivals_intl,Seats_arrivals_total,Total
Seats_arrivals_intl,1.000000,0.826885,0.959151
Seats_arrivals_total,0.826885,1.000000,0.716064
Total,0.959151,0.716064,1.000000


In [214]:
vu_cc = cross_corr_df(vu_merged[incl_cols], "Seats_arrivals_intl", "Total")
vu_cc.head(5)

NameError: name 'vu_merged' is not defined

### Stationarity

In [22]:
get_adf_df(vu_merged[incl_cols], incl_cols)

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
Seats_arrivals_intl,-1.266565,0.644327,2.0,39.0,-3.610400,-2.939109,-2.608063
Seats_arrivals_total,-1.576570,0.495352,0.0,41.0,-3.600983,-2.935135,-2.605963
Total,-2.809163,0.056977,10.0,31.0,-3.661429,-2.960525,-2.619319


In [25]:
vu_diff = vu_merged[incl_cols].diff().dropna()
get_adf_df(vu_diff, incl_cols)

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
Seats_arrivals_intl,-3.604892,5.668196e-03,1.0,39.0,-3.610400,-2.939109,-2.608063
Seats_arrivals_total,-6.723542,3.434833e-09,0.0,40.0,-3.605565,-2.937069,-2.606986
Total,-5.075438,1.570404e-05,0.0,40.0,-3.605565,-2.937069,-2.606986


### Granger Causality

In [29]:
grangers_causation_matrix(vu_diff, incl_cols, maxlag=12)

,Seats_arrivals_intl_x,Seats_arrivals_total_x,Total_x
Seats_arrivals_intl_y,1.0,0.00001,0.0
Seats_arrivals_total_y,0.0,1.00000,0.0
Total_y,0.0,0.00000,1.0
